In [ ]:
# Cell 1: Install required packages
!pip install matplotlib seaborn pandas wordcloud --quiet

# Cell 2: Imports
import importlib
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import time

# Reload in case main.py changes
import main
importlib.reload(main)

# Cell 3: Input sample code
code_input = """
fun main() {
    val n = 10
    var a = 0
    var b = 1
    println(a)
    println(b)
    for (i in 2 until n) {
        val c = a + b
        println(c)
        a = b
        b = c
    }
}
"""

# Cell 4: Run Gemini analysis
explainer = main.GeminiCodeExplainer()
results = explainer.explain_code(code_input, add_inline_comments=True)

# Cell 5: Display key results
print("🧠 Model Used:", results["model_used"])
print("📝 Language Detected:", results["language"])
print("\n📖 Overall Explanation:\n", results["overall_explanation"][:500], "...\n")  # Trimmed for display

# Cell 6: Structure block explanations into DataFrame
block_data = []
for block, explanation in results["block_explanations"].items():
    word_count = len(explanation.split())
    char_count = len(explanation)
    block_data.append({
        "block": block,
        "explanation": explanation,
        "word_count": word_count,
        "char_count": char_count
    })

df = pd.DataFrame(block_data)

# Cell 7: Bar Plot – Word Count per Block
plt.figure(figsize=(10,6))
sns.barplot(data=df, x="block", y="word_count", palette="viridis")
plt.title("🧩 Word Count per Code Block Explanation")
plt.xlabel("Block")
plt.ylabel("Word Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Cell 8: Pie Chart – Explanation Word Share
plt.figure(figsize=(8,8))
plt.pie(df['word_count'], labels=df['block'], autopct='%1.1f%%', startangle=140)
plt.title("🔍 Explanation Word Distribution")
plt.axis('equal')
plt.show()

# Cell 9: Code Size Comparison – Original vs Commented
original_len = len(results["original_code"].split())
commented_len = len(results["commented_code"].split())

plt.figure(figsize=(6,4))
plt.bar(["Original", "Commented"], [original_len, commented_len], color=["#1f77b4", "#ff7f0e"])
plt.title("📏 Token Count: Original vs Commented Code")
plt.ylabel("Token Count")
plt.grid(axis='y')
plt.show()

# Cell 10: Heatmap of Block Stats
if len(df) > 1:
    corr = df[["word_count", "char_count"]].corr()
    sns.heatmap(corr, annot=True, cmap="coolwarm")
    plt.title("📊 Correlation Matrix of Block Explanation Stats")
    plt.show()

# Cell 11: Word Cloud from All Explanations
all_text = " ".join(df["explanation"])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(all_text)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("☁️ Word Cloud of Block Explanations")
plt.show()

# Cell 12: Save Results
df.to_csv("gemini_block_explanations.csv", index=False)

with open("gemini_full_output.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ All analysis completed and files saved!")


ModuleNotFoundError: No module named 'wordcloud'